## Uavs Collision

This notebook implements a uav logic instead of a plan and decisions are done online following that logic

In [ ]:
import os
import subprocess
import numpy as np

# Custom modules
from vehicle_logic import VehicleLogic
from helpers.change_coordinates import heading_to_yaw,find_spawns,global2local
from helpers.visualization.gazebo_world import update_world
from helpers.visualization.plots import plot_3d_interactive
from helpers.change_coordinates import local2global
from plan.actions.navegation import get_local_position,make_go_to
from plan import Plan,State
from gcs import GCS

Kill all related process

In [26]:
for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
    os.system(f"pkill -9 -f {process}")

Paths

In [27]:
QGC_path='~/QGroundControl.AppImage'
gazebo_word_path="gazebo_worlds/runway.world"
ardupilot_vehicle_path='~/ardupilot/Tools/autotest/sim_vehicle.py'

## Choose Simulator

In [28]:
simulator ='gazebo' # 'QGroundControl' #

## Create Plans

In [29]:
# local_paths=[global2local(path, home,pairwise=True) for path,home in zip(global_paths,homes)]
# local_paths

In [30]:
offsets = [(-10, 0, 0, 0),(10, 0, 0, 0)]#
n_uavs = len(offsets)


homes=np.array([offset[:3] for offset in offsets])
gcs=GCS(positions=homes.copy())
global_paths=[np.array([[-10, 0, 5],[10, 0, 5]]),
              np.array([[10, 0, 5],[-10, 0, 5]])]

# global_paths=[np.array([[-10, 0, 5],[10, 0, 5]]),
#               np.array([[10, 0, 5],[2, 0, 5]])]
local_paths=[global2local(path, home,pairwise=True) for path,home in zip(global_paths,homes)]
plans=[Plan.basic(wps=path,alt=5,wp_margin=0.5,navegation_speed=1) for path in local_paths]



## Debouging
# plan1=Plan.basic(wps=local_paths[0],alt=5,wp_margin=0.5,navegation_speed=1,verbose=2)
# from plan.actions.pre_arm import make_pre_arm
# from plan.actions.change_mode import make_set_mode
# from plan.actions.arm import make_arm
# from plan.actions.take_off import make_takeoff
# from plan.actions.land import make_land
# from plan.actions.change_parameter import make_change_nav_speed
# from plan.actions.navegation import make_path
# verbose=0
# navegation_speed=1
# alt=5
# wp_margin=0.5


# plan2=Plan('hover')
# plan2.add(make_pre_arm(verbose=verbose))
# if navegation_speed!=5:
#     plan2.add(make_change_nav_speed(speed=navegation_speed,verbose=verbose))
# plan2.add(make_set_mode('GUIDED',verbose=verbose))
# plan2.add(make_arm(verbose=verbose))
# plan2.add(make_takeoff(altitude=alt,wp_margin=wp_margin,verbose=verbose))
# plan2.add(make_path(wps=local_paths[1],wp_margin=wp_margin,verbose=verbose))
# plans=[plan1,plan2]


markers = {f'waypoints': {'pos':global_paths[0],'color':'blue'}}


plot_3d_interactive(markers,title='Simulation Markers',expand=[0.2,0.2,0.6],ground=-0.05)



if simulator == 'QGroundControl':
    home_position=(-35.3633245,149.1652241,0,0)
    spawns=find_spawns(home_position, offsets)

Plan 'basic' created — no connection yet 🧩
Action 'Pre-Arm Check' created — no connection yet 🧩
Step 'Check disarmed' created — no connection yet 🧩
Step 'Check EKF' created — no connection yet 🧩
Step 'Check GPS' created — no connection yet 🧩
Step 'Check system' created — no connection yet 🧩
Action 'Set Mode: GUIDED' created — no connection yet 🧩
Step 'Switch to GUIDED' created — no connection yet 🧩
Action 'Set Navigation Speed' created — no connection yet 🧩
Step 'Set speed to 1.00 m/s' created — no connection yet 🧩
Action 'Arm' created — no connection yet 🧩
Step 'arm' created — no connection yet 🧩
Action 'takeoff' created — no connection yet 🧩
Step 'takeoff' created — no connection yet 🧩
Action 'fly' created — no connection yet 🧩
Step 'go to  -> (0, 0, 5)' created — no connection yet 🧩
Step 'go to  -> (20, 0, 5)' created — no connection yet 🧩
Action 'Land' created — no connection yet 🧩
Step 'land' created — no connection yet 🧩
Plan 'basic' created — no connection yet 🧩
Action 'Pre-Arm 

## Waypoint seletion algorithm

In [31]:
if simulator == 'QGroundControl':
    sim_path = os.path.expanduser(QGC_path)
    sim_cmd =[sim_path]
elif simulator == 'gazebo':
    # Convert to Gazebo format (name, x, y, z, roll, pitch, yaw)
    drones = [(east, north, up, 0, 0, heading_to_yaw(heading)) for i, (east, north, up, heading) in enumerate(offsets)]
    world_path = os.path.expanduser(gazebo_word_path)
    updated_world_path = update_world(drones,markers,world_path)
    sim_cmd = ["gazebo", "--verbose", updated_world_path] 

simulator_process = subprocess.Popen(
                    sim_cmd,
                    stdout=subprocess.DEVNULL,  # Suppress standard output
                    stderr=subprocess.DEVNULL,  # Suppress error output
                    shell=False  # Ensure safety when passing arguments
                    )

Launch Vehicle

In [32]:
vehicle_path = os.path.expanduser(ardupilot_vehicle_path)

In [33]:
for i in range(n_uavs):
    vehicle_cmd = f"python3 {vehicle_path} -v ArduCopter -I{i} --sysid {i+1} --no-rebuild"
    if simulator == 'QGroundControl':
        spawn=','.join(map(str, spawns[i]))
        vehicle_cmd += f" --custom-location={spawn}"
    elif simulator == 'gazebo':
        vehicle_cmd += " -f gazebo-iris"
    subprocess.Popen(["gnome-terminal", "--", "bash", "-c", f"{vehicle_cmd}; exec bash"])

Create UavLogic

In [34]:
uavs=[]
for i,(plan,home) in enumerate(zip(plans,homes)):
    uavs.append(VehicleLogic(sys_id=i+1,
                    home=home,
                    plan= plan))

Vehicle 1: Step 'Check disarmed' is now connected ✅🔗
Vehicle 1: Step 'Check EKF' is now connected ✅🔗
Vehicle 1: Step 'Check GPS' is now connected ✅🔗
Vehicle 1: Step 'Check system' is now connected ✅🔗
Vehicle 1: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 1: Action 'Pre-Arm Check' is now connected ✅🔗
Vehicle 1: Step 'Switch to GUIDED' is now connected ✅🔗
Vehicle 1: Action 'Set Mode: GUIDED' is now connected ✅🔗
Vehicle 1: Action 'Set Mode: GUIDED' is now connected ✅🔗
Vehicle 1: Step 'Set speed to 1.00 m/s' is now connected ✅🔗
Vehicle 1: Action 'Set Navigation Speed' is now connected ✅🔗
Vehicle 1: Action 'Set Navigation Speed' is now connected ✅🔗
Vehicle 1: Step 'arm' is now connected ✅🔗
Vehicle 1: Action 'Arm' is now connected ✅🔗
Vehicle 1: Action 'Arm' is now connected ✅🔗
Vehicle 1: Step 'takeoff' is now connected ✅🔗
Vehicle 1: Action 'takeoff' is now connected ✅🔗
Vehicle 1: Action 'takeoff' is now connected ✅🔗
Vehicle 1: Step 'go to  -> (0, 0, 5)' is now connected ✅🔗
Vehicle 1: 

In [35]:
def get_nearest_neighbor(target_id,positions_dict):
    target_position = np.array(positions_dict[target_id])  # Get the position of the target ID
    
    min_distance = float('inf')
    nearest_id = None

    for other_id, position in positions_dict.items():
        if other_id == target_id:
            continue  # Skip itself
        
        distance = np.linalg.norm(target_position - np.array(position))  # Compute Euclidean distance
        
        if distance < min_distance:
            min_distance = distance
            nearest_id = other_id  # Update nearest ID
    
    return nearest_id,min_distance

In [36]:
print(uavs[0].plan)

🚀 <Plan 'basic'>
  🕓 <Action 'Pre-Arm Check'>
    🕓 <Step 'Check disarmed'>
    🕓 <Step 'Check EKF'>
    🕓 <Step 'Check GPS'>
    🕓 <Step 'Check system'>
  🕓 <Action 'Set Mode: GUIDED'>
    🕓 <Step 'Switch to GUIDED'>
  🕓 <Action 'Set Navigation Speed'>
    🕓 <Step 'Set speed to 1.00 m/s'>
  🕓 <Action 'Arm'>
    🕓 <Step 'arm'>
  🕓 <Action 'takeoff'>
    🕓 <Step 'takeoff'>
  🕓 <Action 'fly'>
    🕓 <Step 'go to  -> (0, 0, 5)'>
    🕓 <Step 'go to  -> (20, 0, 5)'>
  🕓 <Action 'Land'>
    🕓 <Step 'land'>


In [37]:
def get_avoidance_wp(pos: np.ndarray,
                    obj_pos:np.ndarray,
                    goal_pos:np.ndarray,
                    direction: str = 'left'):
    """
    Sends a velocity command in body frame, orthogonal to the direction of wp.
    `direction` can be 'left' or 'right' (relative to wp direction).
    """

    # Normalize wp direction (ignore Z)
    obj_dir = (obj_pos -pos)[:2]
    goal_dir = (goal_pos -pos)[:2]
    if np.dot(obj_dir,goal_dir)<0:
        return goal_pos
    obj_dir = obj_dir / np.linalg.norm(obj_dir)
    # Get orthogonal direction
    if direction == 'left':
        ortho = np.array([-obj_dir[1], obj_dir[0],0])
    elif direction == 'right':
        ortho = np.array([obj_dir[1], -obj_dir[0],0])
    else:
        raise ValueError("Direction must be 'left' or 'right'")

    # Scale to desired speed
    return pos+ortho
   

In [38]:
uavs[0].current_step().state

'NOT_STARTED'

In [39]:
uavs[1].current_action().name

'Pre-Arm Check'

In [40]:
gcs.incomplete_missions

{1, 2}

## Execute Plan

In [41]:



global_poss = {}
collision_modes=n_uavs*[False]
while gcs.incomplete_missions:
    ## Get global positions
    for id in gcs.incomplete_missions:
        i = id-1
        uav=uavs[i]
        current_action=uav.current_action().name 
        if current_action in ['takeoff','fly']:
            local_pos=get_local_position(uav.conn)
            if local_pos is not False:
                global_poss[i] = local2global(local_pos,uav.home,pairwise=True)
    to_remove = set()
    for id in gcs.incomplete_missions:
        i = id-1
        uav = uavs[i]
        if i in global_poss.keys():
            j,min_dist=get_nearest_neighbor(target_id=i,positions_dict=global_poss)
            if min_dist<5 and not collision_modes[i]:  
                collision_modes[i]=True
                wp=uav.plan.current.current.wp
                print('waypoint',wp)
                next_wp = get_avoidance_wp(pos=global2local(global_poss[i],uav.home,pairwise=True),
                                           obj_pos=global2local(global_poss[j],uav.home,pairwise=True),
                                           goal_pos=wp)
                print(next_wp)
                next_step=make_go_to(wp=next_wp,wp_margin=0.5,verbose=2,cause_text='(avoidance)')
                next_step.bind_connection(uav.conn)
                uav.plan.current.add_now(next_step)
            if collision_modes[i] and uav.current_step().state==State.DONE:
                collision_modes[i] = False
                print(collision_modes[i])
        uav.plan.act()
        if uav.plan.state == State.DONE:
            to_remove.add(id)
            
    gcs.update_missions(to_remove)

Vehicle 1: ▶️ Starting Step: Check disarmed
Vehicle 2: ▶️ Starting Step: Check disarmed
Vehicle 1: ✅ Step: Check disarmed is done
Vehicle 2: ✅ Step: Check disarmed is done
Vehicle 1: ▶️ Starting Step: Check EKF
Vehicle 2: ▶️ Starting Step: Check EKF
Vehicle 2: ✅ Step: Check EKF is done
Vehicle 2: ▶️ Starting Step: Check GPS
Vehicle 1: ✅ Step: Check EKF is done
Vehicle 1: ▶️ Starting Step: Check GPS
Vehicle 2: ✅ Step: Check GPS is done
Vehicle 2: ▶️ Starting Step: Check system
Vehicle 1: ✅ Step: Check GPS is done
Vehicle 1: ▶️ Starting Step: Check system
Vehicle 2: ✅ Step: Check system is done
Vehicle 2: ▶️ Starting Step: Switch to GUIDED
Vehicle 2: ✅ Step: Switch to GUIDED is done
Vehicle 2: ▶️ Starting Step: Set speed to 1.00 m/s
Vehicle 2: ✅ Step: Set speed to 1.00 m/s is done
Vehicle 2: ▶️ Starting Step: arm
Vehicle 2: ✅ Step: arm is done
Vehicle 2: ▶️ Starting Step: takeoff
Vehicle 2:📍 Distance to target: 5.03 m
Vehicle 1: ✅ Step: Check system is done
Vehicle 2:📍 Distance to target

In [45]:
uavs[0].plan

✅ <Plan 'basic'>
  ✅ <Action 'Pre-Arm Check'>
    ✅ <Step 'Check disarmed'>
    ✅ <Step 'Check EKF'>
    ✅ <Step 'Check GPS'>
    ✅ <Step 'Check system'>
  ✅ <Action 'Set Mode: GUIDED'>
    ✅ <Step 'Switch to GUIDED'>
  ✅ <Action 'Set Navigation Speed'>
    ✅ <Step 'Set speed to 1.00 m/s'>
  ✅ <Action 'Arm'>
    ✅ <Step 'arm'>
  ✅ <Action 'takeoff'>
    ✅ <Step 'takeoff'>
  ✅ <Action 'fly'>
    ✅ <Step 'go to  -> (0, 0, 5)'>
    ✅ <Step 'go to (avoidance) -> (7.389691582256462, 1.0166675906866414, 4.886508941650391)'>
    ✅ <Step 'go to (avoidance) -> (8.056342801816355, 1.6325860411148836, 4.884792327880859)'>
    ✅ <Step 'go to (avoidance) -> (8.558771291640262, 2.3344267307088984, 4.883833408355713)'>
    ✅ <Step 'go to (avoidance) -> (9.568935114873119, 2.385972133763109, 4.883977890014648)'>
    ✅ <Step 'go to (avoidance) -> (10.03502741763638, 2.615447702207187, 4.884220123291016)'>
    ✅ <Step 'go to  -> (20, 0, 5)'>
  ✅ <Action 'Land'>
    ✅ <Step 'land'>

In [46]:
uavs[1].plan

✅ <Plan 'basic'>
  ✅ <Action 'Pre-Arm Check'>
    ✅ <Step 'Check disarmed'>
    ✅ <Step 'Check EKF'>
    ✅ <Step 'Check GPS'>
    ✅ <Step 'Check system'>
  ✅ <Action 'Set Mode: GUIDED'>
    ✅ <Step 'Switch to GUIDED'>
  ✅ <Action 'Set Navigation Speed'>
    ✅ <Step 'Set speed to 1.00 m/s'>
  ✅ <Action 'Arm'>
    ✅ <Step 'arm'>
  ✅ <Action 'takeoff'>
    ✅ <Step 'takeoff'>
  ✅ <Action 'fly'>
    ✅ <Step 'go to  -> (0, 0, 5)'>
    ✅ <Step 'go to (avoidance) -> (-7.634991875225212, -1.023665012070499, 4.91607666015625)'>
    ✅ <Step 'go to (avoidance) -> (-8.306297979123483, -1.6529991418343355, 4.914173603057861)'>
    ✅ <Step 'go to (avoidance) -> (-8.828567663100223, -2.372725099162878, 4.904059886932373)'>
    ✅ <Step 'go to (avoidance) -> (-9.810314710725553, -2.432905173115851, 4.903961658477783)'>
    ✅ <Step 'go to (avoidance) -> (-10.264731017268158, -2.7237859975932888, 4.900784969329834)'>
    ✅ <Step 'go to  -> (-20, 0, 5)'>
  ✅ <Action 'Land'>
    ✅ <Step 'land'>

## Kill all related process

In [44]:
# for process in ["QGroundControl", "sim_vehicle.py", "arducopter", "gazebo", "mavproxy"]:
#     os.system(f"pkill -9 -f {process}")